In [8]:
include("../src/Julia.jl")

objc[54423]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[54423]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[54423]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[54423]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


pcgWorker (generic function with 1 method)

In [9]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [10]:
graphp = "../../graphs/rand_iu_1000_3/"

"../../graphs/rand_iu_1000_3/"

In [11]:
a = readFromFile(graphp * "graph.mtx");

In [12]:
tree = []
for i in 1:3
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [13]:
x = readFromFile(graphp * "x.vec");

In [14]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [15]:
numIts = 200;

for i in 1:3
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  0.358452 seconds (275.40 k allocations: 11.698 MB)
  0.388139 seconds (123.84 k allocations: 4.983 MB, 2.13% gc time)
  1.168069 seconds (3.17 M allocations: 95.401 MB, 2.48% gc time)
Working on iteration 10 with a-norm error 9.223829307417224
Working on iteration 20 with a-norm error 3.56355959116662
Working on iteration 30 with a-norm error 1.272990115017813
Working on iteration 40 with a-norm error 0.4258251578984389
Working on iteration 50 with a-norm error 0.10949194729002161
Working on iteration 60 with a-norm error 0.0360461066110392
Working on iteration 70 with a-norm error 0.009459512308796198
Working on iteration 80 with a-norm error 0.0033195954420127704
Working on iteration 90 with a-norm error 0.001024993144426708
Working on iteration 100 with a-norm error 0.00033846464899116134
Working on iteration 110 with a-norm error 8.349008610190722e-5
Working on iteration 120 with a-norm error 2.5117889695710093e-5
Working on iteration 130 with a-norm error 8.290268984740628e-6
Wo